<a href="https://colab.research.google.com/github/TheLazyCactus/ML_Project/blob/main/Final_with_lag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


url = 'https://raw.githubusercontent.com/TheLazyCactus/ML_Project/refs/heads/main/ML_Project_safety.csv'
df = pd.read_csv(url, sep=";", low_memory =False)

Need to change the order to get the oldest value first

In [70]:
df = df.sort_values(by="Year", ascending=True)
#Drop columns
cols_to_remove = ["FAR", "TRIR total","TRIR company only", "TRIR contractor only", "LTIR company only", "LTIR contractor only"]
df.drop(["FAR total", "LTIR company only","LTIR contractor only","TRIR company only", "TRIR contractor only"], axis = 1, inplace=True)

df = df.drop(columns=[col for col in cols_to_remove if col in df.columns])  # List of columns to convert
cols = ["LTIR total"]
cols_to_process = [col for col in cols if col in df.columns]

# Replace commas with dots and convert to float for the existing columns
df[cols_to_process] = df[cols_to_process].replace(',', '.', regex=True).astype(float)
df[cols] = df[cols].astype(float)

df = df.fillna(0)



#df_total = df[["Year", "Company code", "TRIR total", "LTIR total"]]
#df_company = df[["Year", "Company code", "TRIR company only", "LTIR company only"]]
#df_contractor = df[["Year", "Company code", "TRIR contractor only", "LTIR contractor only"]]

**EDA**

In [69]:
print(df.shape)
print(df.columns)
print(df.dtypes)
print(df.nunique())
print(df.isna().sum())

(487, 4)
Index(['Year', 'Company code', 'TRIR total', 'LTIR total'], dtype='object')
Year              int64
Company code     object
TRIR total      float64
LTIR total      float64
dtype: object
Year             10
Company code     59
TRIR total      247
LTIR total      121
dtype: int64
Year            0
Company code    0
TRIR total      0
LTIR total      0
dtype: int64


One hot encoding

In [71]:
from sklearn.preprocessing import OneHotEncoder

# Ensure the "Year" column exists and filter only the last 3 years
year_column = "Year"  # Adjust this if your column name is different
if year_column not in df.columns:
    raise KeyError(f"Column '{year_column}' not found. Available columns: {df.columns}")

df_filtered = df[df[year_column].isin([2020, 2021, 2022, 2023])]

# Display filtered data
print(df_filtered.head())

     Year Company code  LTIR total
167  2020   N                 0.41
168  2020   O                 0.39
169  2020   P                 0.37
170  2020   Q                 0.34
173  2020   T                 0.26


In [72]:
# Create lag features: LTIR from the past 1 2 and 3 years
df_filtered['LTIR_1'] = df_filtered.groupby('Company code')['LTIR total'].shift(1)
df_filtered['LTIR_2'] = df_filtered.groupby('Company code')['LTIR total'].shift(2)
df_filtered['LTIR_3'] = df_filtered.groupby('Company code')['LTIR total'].shift(3)
df_filtered = df_filtered.rename(columns={"LTIR total": "LTIR 2023", "LTIR_1": "LTIR 2022", "LTIR_2": "LTIR 2021", "LTIR_3": "LTIR 2020"})
# Drop rows with NaN values (first 2 years for each company)
data = df_filtered.dropna().reset_index(drop=True)
print(data)  # Fixed column name

# Drop rows with NaN values due to shifting
df_filtered = df_filtered.dropna().reset_index(drop=True)  # Ensure you are dropping from df_final, not df

    Year Company code  LTIR 2023  LTIR 2022  LTIR 2021  LTIR 2020
0   2023   Q                0.51       0.44       0.38       0.34
1   2023   R                0.51       0.34       0.38       0.31
2   2023   Overall          0.24       0.28       0.22       0.22
3   2023   W                0.30       0.26       0.23       0.25
4   2023   X                0.28       0.24       0.23       0.24
5   2023   Y                0.27       0.23       0.21       0.24
6   2023   P                0.51       0.45       0.40       0.37
7   2023   V                0.33       0.27       0.27       0.25
8   2023   O                0.54       0.53       0.41       0.39
9   2023   C                1.22       1.08       0.99       1.04
10  2023   M                0.55       0.56       0.45       0.44
11  2023   L                0.59       0.58       0.46       0.47
12  2023   K                0.63       0.63       0.66       0.50
13  2023   J                0.67       0.70       0.68       0.50
14  2023  

C:\Users\UTILISATEUR\AppData\Local\Temp\ipykernel_34940\3061365396.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['LTIR_1'] = df_filtered.groupby('Company code')['LTIR total'].shift(1)
C:\Users\UTILISATEUR\AppData\Local\Temp\ipykernel_34940\3061365396.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['LTIR_2'] = df_filtered.groupby('Company code')['LTIR total'].shift(2)
C:\Users\UTILISATEUR\AppData\Local\Temp\ipykernel_34940\3061365396.py:4: SettingWithCopyWarning: 
A value is

In [73]:
from sklearn.preprocessing import OneHotEncoder


# Ensure "Company code" exists before encoding
company_column = "Company code"
if company_column not in df_filtered.columns:
    raise KeyError(f"Column '{company_column}' not found. Available columns: {df_filtered.columns}")

# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Fit-transform the 'Company code' column
encoded_company = encoder.fit_transform(df_filtered[[company_column]])

# Get feature names for encoded categories
company_columns = encoder.get_feature_names_out([company_column])

# Convert to DataFrame
df_encoded = pd.DataFrame(encoded_company, columns=company_columns)

# Merge encoded data back into df_filtered and drop the original "Company code"
df_final = pd.concat([df_filtered.reset_index(drop=True), df_encoded], axis=1)
df_final.drop(columns=[company_column], inplace=True, errors='ignore')

import pickle
with open('encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)


In [75]:
df_final.drop(columns=["Year"], inplace=True)

In [76]:
df_final

,LTIR 2023,LTIR 2022,LTIR 2021,LTIR 2020,Company code_A,Company code_AA,Company code_B,Company code_BB,Company code_C,Company code_CC,...,Company code_QQ,Company code_R,Company code_RR,Company code_SS,Company code_TT,Company code_UU,Company code_V,Company code_W,Company code_X,Company code_Y
0,0.51,0.44,0.38,0.34,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.51,0.34,0.38,0.31,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.24,0.28,0.22,0.22,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.30,0.26,0.23,0.25,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.28,0.24,0.23,0.24,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0.27,0.23,0.21,0.24,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,0.51,0.45,0.40,0.37,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.33,0.27,0.27,0.25,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,0.54,0.53,0.41,0.39,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1.22,1.08,0.99,1.04,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Future values

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Check that df_final has the expected columns
required_columns = ['LTIR 2020', 'LTIR 2021', 'LTIR 2022', 'LTIR 2023']
missing_columns = [col for col in required_columns if col not in df_final.columns]
if missing_columns:
    raise ValueError(f"Missing columns: {missing_columns}")

# Features for initial model training
X = df_final[['LTIR 2020', 'LTIR 2021', 'LTIR 2022']]  # Predictors
y = df_final['LTIR 2023']  # Target variable

# Train Linear Regression model
model = LinearRegression()
linear_reg = model.fit(X, y)

# Prepare storage for future predictions
future_data = []

# Iterate through each row in the dataframe to predict future LTIR
for _, row in df_final.iterrows():
    # Extract relevant lag features
    ltir_2021 = row['LTIR 2021']
    ltir_2022 = row['LTIR 2022']
    ltir_2023 = row['LTIR 2023']

    # Predict 2024 LTIR using LTIR 2021, LTIR 2022, and LTIR 2023
    input_2024 = pd.DataFrame([[ltir_2021, ltir_2022, ltir_2023]], columns=['LTIR 2020', 'LTIR 2021', 'LTIR 2022'])
    predicted_2024 = linear_reg.predict(input_2024)[0]

    # Predict 2025 LTIR using LTIR 2022, LTIR 2023, and predicted 2024
    input_2025 = pd.DataFrame([[ltir_2022, ltir_2023, predicted_2024]], columns=['LTIR 2020', 'LTIR 2021', 'LTIR 2022'])
    predicted_2025 = linear_reg.predict(input_2025)[0]

    # Predict 2026 LTIR using LTIR 2023, predicted 2024, and predicted 2025
    input_2026 = pd.DataFrame([[ltir_2023, predicted_2024, predicted_2025]], columns=['LTIR 2020', 'LTIR 2021', 'LTIR 2022'])
    predicted_2026 = linear_reg.predict(input_2026)[0]

    # Append predictions to the future_data list
    future_data.append([predicted_2024, predicted_2025, predicted_2026])

# Convert predictions into a DataFrame
future_df = pd.DataFrame(future_data, columns=['LTIR 2024', 'LTIR 2025', 'LTIR 2026'])

# Concatenate predictions with original dataset
df_final = pd.concat([df_final, future_df], axis=1)

# Display the updated DataFrame
print("Updated DataFrame with future LTIR values:")
print(df_final)


Updated DataFrame with future LTIR values:
    LTIR 2023  LTIR 2022  LTIR 2021  LTIR 2020  Company code_A           \
0        0.51       0.44       0.38       0.34                      0.0   
1        0.51       0.34       0.38       0.31                      0.0   
2        0.24       0.28       0.22       0.22                      0.0   
3        0.30       0.26       0.23       0.25                      0.0   
4        0.28       0.24       0.23       0.24                      0.0   
5        0.27       0.23       0.21       0.24                      0.0   
6        0.51       0.45       0.40       0.37                      0.0   
7        0.33       0.27       0.27       0.25                      0.0   
8        0.54       0.53       0.41       0.39                      0.0   
9        1.22       1.08       0.99       1.04                      0.0   
10       0.55       0.56       0.45       0.44                      0.0   
11       0.59       0.58       0.46       0.47           

In [83]:
df_final.columns

Index(['LTIR 2023', 'LTIR 2022', 'LTIR 2021', 'LTIR 2020',
       'Company code_A         ', 'Company code_AA        ',
       'Company code_B         ', 'Company code_BB        ',
       'Company code_C         ', 'Company code_CC        ',
       'Company code_D         ', 'Company code_DD        ',
       'Company code_E         ', 'Company code_EE        ',
       'Company code_F         ', 'Company code_FF        ',
       'Company code_G         ', 'Company code_H         ',
       'Company code_HH        ', 'Company code_I         ',
       'Company code_II        ', 'Company code_J         ',
       'Company code_K         ', 'Company code_KK        ',
       'Company code_L         ', 'Company code_LL        ',
       'Company code_M         ', 'Company code_MM        ',
       'Company code_N         ', 'Company code_NN        ',
       'Company code_O         ', 'Company code_OO        ',
       'Company code_Overall   ', 'Company code_P         ',
       'Company code_PP   

In [82]:
df_final.to_csv("final_LTIR_predictions.csv", index=False)


In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Create lag features (past 3 years of LTIR & TRIR)
for lag in range(1, 4):
    df_final[f'LTIR_lag_{lag}'] = df_final['LTIR total'].shift(lag)
    df_final[f'TRIR_lag_{lag}'] = df_final['TRIR total'].shift(lag)  # Fixed column name

# Drop rows with NaN values due to shifting
df_final = df_final.dropna().reset_index(drop=True)  # Ensure you are dropping from df_final, not df

# Features (X) and Target (y)
X = df_final.drop(columns=["Year", "LTIR total"])  # Features include company code (one-hot) + past LTIR/TRIR
y = df_final["LTIR total"]  # Target: LTIR

# Train-test split (using last 3 years for testing)
#train_size = len(X) - 3
#X_train, X_test = X[:train_size], X[train_size:]
#y_train, y_test = y[:train_size], y[train_size:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)





***LTIR Prediction***

**With lag**

In [5]:
forest = RandomForestRegressor(n_estimators=200,
                             max_depth=10, random_state=42)
forest.fit(X_train, y_train)
pred = forest.predict(X_test)

print("forest score")
print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test))
print("R2 score (Test)", forest.score(X_test, y_test))
print("R2 score (Train)", forest.score(X_train, y_train))

bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=10),
                               n_estimators=100,
                               max_samples=100,
                                random_state=42)#take 80% of the total samples

bagging_reg.fit(X_train, y_train)
pred = bagging_reg.predict(X_test)

print("Bag score")
print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test))
print("R2 score", bagging_reg.score(X_test, y_test))

gb_reg = GradientBoostingRegressor(max_depth=10,
                                   n_estimators=100,
                                   random_state=42)
gb_reg.fit(X_train, y_train)
pred = gb_reg.predict(X_test)

print("GradBoost score")
print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test))
print("R2 score", gb_reg.score(X_test, y_test))
print("R2 score", gb_reg.score(X_train, y_train))

ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=10),
                            n_estimators=100,
                            random_state=42)
ada_reg.fit(X_train, y_train)
pred = ada_reg.predict(X_test)
print("AdaBoost score")
print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test))
print("R2 score", ada_reg.score(X_test, y_test))
print("R2 score", ada_reg.score(X_train, y_train))

forest score
MAE 0.06652994931629752
RMSE 0.01390703707592422
R2 score (Test) 0.838373186203105
R2 score (Train) 0.9520094999782749
Bag score
MAE 0.06568040410306535
RMSE 0.013703926933275986
R2 score 0.8407337210191729
GradBoost score
MAE 0.0723779449124633
RMSE 0.028402719613222743
R2 score 0.6699051674925738
R2 score 0.999999999265823
AdaBoost score
MAE 0.045306451612903226
RMSE 0.005869040322580647
R2 score 0.9317903387899632
R2 score 0.9986260603162677


Ada boost seems the best model

**Cross validation**

In [94]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation
cv_results = cross_val_score(ada_reg, X, y, cv=5, scoring='neg_mean_absolute_error')

# cv_results returns the negative of the score (since lower MAE is better)
# Convert it back to positive values
cv_results = -cv_results

# Print the results
print("ada boost cross validation")
print(f"Cross-validation results for MAE: {cv_results}")
print(f"Average MAE: {np.mean(cv_results)}")
print(f"Standard Deviation of MAE: {np.std(cv_results)}")

ada boost cross validation
Cross-validation results for MAE: [0.09888958 0.08773413 0.061      0.16586207 0.05      ]
Average MAE: 0.09269715585659552
Standard Deviation of MAE: 0.040605022180807616


**Overfitting?**

Compare MAE cross validation to train MAE
If cross validation MAE too high compare to train MAE, model good for training sata but not for new data -> Overfitting

**Hyper param tuning**

In [ ]:
grid = {"n_estimators": [50, 100, 200,500],
        "estimator__max_leaf_nodes": [250, 500, 1000, None],
        "estimator__max_depth":[10,30,50]}

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
random_reg = RandomForestRegressor(DecisionTreeRegressor())
model = GridSearchCV(estimator = random_reg, param_grid = grid, cv=5)
model.fit(X_train, y_train)

In [ ]:
model.best_params_

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error
pred = random_reg.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", root_mean_squared_error(pred, y_test))
print("R2 score", random_reg.score(X_test, y_test))

Hyper param for adaboost

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_absolute_error

# Define the parameter grid for GridSearchCV

param_grid = {"n_estimators": [50, 100, 200,500],
        "estimator__max_leaf_nodes": [250, 500, 1000, None],
        "estimator__max_depth":[10,30,50]}
# Define the AdaBoost model with a DecisionTreeRegressor as base estimator
ada_model = AdaBoostRegressor(
    estimator=DecisionTreeRegressor(),  # Decision tree regressor as base estimator
    random_state=42
)

# Create GridSearchCV, using MAE as the evaluation metric for regression
grid_search = GridSearchCV(ada_model, param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)

# Fit the model with the best parameters
grid_search.fit(X_train, y_train)

# Get the best parameters and the best cross-validation score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validation MAE: {-grid_search.best_score_}")  # Negating the MAE (since it's returned negative)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Print test set MAE (Mean Absolute Error) as the evaluation metric
print(f"Test Set MAE: {mean_absolute_error(y_test, y_pred)}")


Best Parameters: {'estimator__max_depth': 10, 'estimator__max_leaf_nodes': None, 'n_estimators': 100}
Best Cross-Validation MAE: 0.12166203703703701
Test Set MAE: 0.045306451612903226


In [7]:
print("R2 score", best_model.score(X_test, y_test))
print("R2 score", best_model.score(X_train, y_train))

R2 score 0.9317903387899632
R2 score 0.9986260603162677


***Save the model***

In [8]:
import pickle
from sklearn.linear_model import LinearRegression
import numpy as np



# Save the trained model to a file
with open("LTIR_prediction_lag.pkl", "wb") as file:
    pickle.dump(best_model, file)

print("Model saved as model.pkl")

Model saved as model.pkl
